In [1]:
import numpy as np
import matplotlib.pyplot as plt
import emcee
import corner
import scipy
import random
import pandas as pds
from scipy.optimize import minimize


In [2]:
def poissant_distribution(n, lambda_val):
    return np.exp(-lambda_val) * (lambda_val**n) / np.math.factorial(n)

def negative_probability_distribution(lambda_val, n):
    return -poissant_distribution(n, lambda_val)

def maximize_distribution(n):
    result = scipy.optimize.minimize(negative_probability_distribution, x0=[10], bounds=[(0,None)], args=(n,))
    return result.x

def rand_bins():
    numbers = set()
    while len(numbers) < 3:
        numbers.add(random.randint(0, 71))
    return numbers

bin1, bin2, bin3 = rand_bins()

In [3]:
#importing and sorting data
ge_data = pds.read_csv("SuperCDMS/PhysRevD.99.062001-data/MarchAprilFinal.txt", skiprows=1, \
                         names=['time', 'blah'], \
                         delim_whitespace=False
                     )

ge_data = ge_data.sort_values(by='time')

t = np.asarray(ge_data["time"], dtype=np.float32)

In [4]:
min_t=np.min(t)

#construct histogram from our data, every bin is roughly 12 hours
counts, bins = np.histogram(t-min_t,bins=72)
thing = (bins[:-1]+bins[1:])/2
error1 = [0.00,0.37,0.74,1.10,2.34,2.75,3.82,4.25,5.30,6.33,6.78,7.81,8.83,9.28]
error2 = [1.29,2.75,4.25,5.30,6.78,7.81,9.28,10.30,11.32,12.79,13.81,14.82,16.29,17.30]
ntot_plus = np.zeros(np.shape(counts))
ntot_minus = np.zeros(np.shape(counts))
for i,ncount in enumerate(counts):
    if ncount<=20:
        ntot_plus[i] = error2[ncount]-ncount
        ntot_minus[i] = ncount-error1[ncount]
    else:
        ntot_plus[i] = np.sqrt(ncount)
        ntot_minus[i] = np.sqrt(ncount)

In [5]:
lambda_array = []
for i in range(len(counts)):
    result=maximize_distribution(counts[i])
    lambda_array.extend(result)

prob_array = []
for i in range(len(lambda_array)):
    result=poissant_distribution(counts[i], lambda_array[i])
    prob_array.extend([result])

counts_array=np.asarray(counts)
print(counts_array[bin1])

4


In [6]:
def log_likelihood_func(theta, n1, n2, n3): 
    lambda1, lambda2, lambda3 = theta
    sterling1, sterling2, sterling3 = 1 #sterling approximation for large values of n so the factorial function does not run into problems
    if n1 > 20: 
        sterling1 = n1*np.log(n1)-n1
    else:
        sterling1 = np.log(np.math.factorial(n1))
        
    if n2 > 20: 
        sterling2 = n2*np.log(n2)-n2
    else:
        sterling2 = np.log(np.math.factorial(n2))
    
    if n3 > 20: 
        sterling3 = n3*np.log(n3)-n3
    else:
        sterling3 = np.log(np.math.factorial(n3))

    model = (-lambda1 + n1*np.log(lambda1)-sterling1)+(-lambda2 + n2*np.log(lambda2)-sterling2)+(-lambda3 + n3*np.log(lambda3)-sterling3) 
    return model


nll = lambda *args: -log_likelihood_func(*args)
initial = np.asarray([lambda_array[bin1]]) #initial parameter vals? 
soln = scipy.optimize.minimize(nll, initial, args=(counts_array[bin1], counts_array[bin2], counts_array[bin3])) 
#future bugs you probably cannot do it this way, these arguments aren't for one function you're minimizing theyre for three different bins
#do this whole block for all three maybe?
lambda1_ml, lambda2_ml, lambda3_ml = soln.x
print(soln.x)

ValueError: not enough values to unpack (expected 3, got 1)